# 코드 시작 전 설명
1. 음성 데이터에 대한 이론 및 과정을 잘 모르시는 분은 코드를 작성하기 전 overview를 한번 정독하시는 것을 권합니다.

2. 스켈레톤 코드가 적혀있는 해당 ipynb 파일을 다운받아 자신의 작업공간(코랩, 캐글노트북 등등)에서 코드를 작성합니다.

3. 각 코드에는 빈칸과 함께 구현 가이드 라인이 제공됩니다. 해당 가이드라인을 따라 코드를 작성하여 베이스 라인 성능을 달성합니다.
   진행에 어려움이 있으신 경우에는 feature 가공에 사용되는 함수의 documentation 설명을 읽으시는 것을 권합니다.
   
4. (선택)베이스라인에 도달하였다면, 음성 feature를 새롭게 가공하여 추가적인 성능 향상을 달성해봅니다.

In [37]:
import pandas as pd
import numpy as np
import sklearn
import os
from os.path import join

In [38]:
# DATA Load

DATA_PATH = "../input/2022-ml-project3"
# (참고) os.path.join 함수는 여러 문자열을 경로에 대한 문자열로 합쳐주는 함수입니다.
pd_train = pd.read_csv(join(DATA_PATH, 'train_data.csv'))
pd_test = pd.read_csv(join(DATA_PATH, 'test_data.csv'))

print(pd_train.info(), pd_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         1008 non-null   int64 
 1   file_name  1008 non-null   object
 2   emotion    1008 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         432 non-null    int64 
 1   file_name  432 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.9+ KB
None None


# **Feature 추출**

* extract_feature 함수는 음성 데이터를 읽어서 다음의 과정들을 거치며 feature를 추출합니다.(함수(입력) --> 출력)
 1. Sampling&Quantization(continuos audio signal) --> Discrete audio signal
 2. Short Time Fourier Transfrom(Discrete audio signal) --> Spectrogram
 3. Mel-Filter(Spectrogram) --> Mel-Spectrogram
 4. Discrete Cosine Transform(Mel-Spectrogram) --> Mel Frequency Cepstrum Coefficient
 
위에 과정들은 모두 librosa 라이브러리에서 제공하는 함수들을 통하여 구현가능하며 사용되는 함수는 다음과 같습니다.

1. librosa.load : Continuos audio signal(.wav파일)을 Discrete audio signal로 읽음. 
2. librosa.stft : Discrte audio signal을 windowing하여 프레임별로 나눈 후 FFT(Fast Fourier Transform)을 수행
[stft_documentation](https://librosa.org/doc/latest/generated/librosa.stft.html?highlight=stft)
3. librosa.feature.melspectrogram : stft 함수를 통하여 구한 spectrogram을 입력으로 Mel-filter를 적용함으로써 Mel-Spectrogram 생성[melspectrogram_documentation](https://librosa.org/doc/latest/generated/librosa.feature.melspectrogram.html?highlight=melspectrogram)
4. librosa.feature.mfcc : melspectrogram 함수를 통하여 구한 Mel-spectrogram을 입력으로 DCT를 수행함으로써 MFCC를 생성 [mfcc_documentation](https://librosa.org/doc/latest/generated/librosa.feature.mfcc.html?highlight=mfcc)

In [39]:
import librosa
import glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import librosa, librosa.display 


# -------------------------------------
# extract_feature(file_name): <= 코드를 추가하여 음성 feature를 추출하는 코드를 완성하세요
# -------------------------------------
# 목적: MFCC를 비롯한 여러 음성 feature를 추출
# 입력인자: .wav 음성 파일의 이름
# 출력인자: 입력 파일을 가공한 feature들 (Spectrogram, Mel-spectrogram, MFCC)
# -------------------------------------


def extract_feature(file_name):
    
    
    result=np.array([])
    X, sample_rate = librosa.load(file_name, sr=22050)
    specto=abs(librosa.stft(y=X,n_fft=512))
   
    
    spectrogram_feature =np.mean(specto,axis=1)

    power_specto=specto**2
    mel=librosa.feature.melspectrogram(S=power_specto)
    mel_db=librosa.amplitude_to_db(S=mel)
    mel_spectrogram_feature=np.mean(mel_db,axis=1)
   
    mfcc=librosa.feature.mfcc(S=mel_db)
    mfcc_feature =np.mean(mfcc,axis=1)
    #-------------------------------------------------------------------------------

    return spectrogram_feature, mel_spectrogram_feature, mfcc_feature 

# **데이터 불러오기**

### csv파일에 저장된 파일 이름과 학습용 label을 로드하여 학습 및 평가용 데이터를 불러오세요.

In [40]:
#DataFlair - Load the data and extract features for each sound file
# (참고) tqdm 은 진행률에 대한 프로그레스바를 알려주는 라이브러리입니다. 
# for문에 자주 사용되며, 얼마나 진행되었는지를 시각적으로 확인할 수 있어 자주 사용됩니다.
from tqdm.notebook import tqdm
def load_data(data_info, isTrain=True):
    
    PATH = join('/kaggle','input','2022-ml-project3')
    if isTrain:
        train_data = {'spectrogram':[],'mel':[],'mfcc':[]}#음성 feature들을 담는 dictionary
        train_label = []#학습에 사용할 label을 담는 list
        
        file_list = data_info['file_name']
        emotion_list = data_info['emotion']
        # (참고) zip을 통해 2개 이상의 변수를 한 번에 넘길 수 있습니다.
        # 궁금한 점이 있으면 zip() 내장함수를 검색해보시기 바랍니다
        for file_name, emotion in tqdm(zip(file_list, emotion_list)):
            train_path=os.path.join(PATH,'train_data','train_data',file_name)
            
            sp,mel,mfcc=extract_feature(train_path)
            train_data['spectrogram'].append(sp)
            train_data['mel'].append(mel)
            train_data['mfcc'].append(mfcc)
            
            train_label.append(emotion)
            
            
            #----------------------------------------------------------------------------------------- 
            
        return train_data, np.array(train_label)
    
    else:
        test_data = {'spectrogram':[],'mel':[],'mfcc':[]}#음성 feature들을 담는 dictionary
        file_list = data_info['file_name']
    
        for file_name in tqdm(file_list):
            test_path=os.path.join(PATH,'test_data','test_data',file_name)
            
            sp,mel,mfcc=extract_feature(test_path)
            test_data['spectrogram'].append(sp)
            test_data['mel'].append(mel)
            test_data['mfcc'].append(mfcc)
            
            
            
            #----------------------------------------------------------------------------------------- 
            
        return test_data

#DataFlair - Split the dataset
train_data, y_train = load_data(pd_train)
test_data = load_data(pd_test, isTrain=False)

0it [00:00, ?it/s]

  0%|          | 0/432 [00:00<?, ?it/s]

# 모델 학습 및 추론
위에서 우리는 
1. 음성 신호에 대하여 Short Time Fourier Transform(stft)를 통해 **spectrogram**을 구하고,
2. 구한 spectrogram에 mel-filter를 씌워 **mel-spectrogram**을 구했으며,
3. 마지막으로 mel-spectrogram에 Discrete cosine transform(DCT) 취해줌으로써 **mfcc**까지 계산하였습니다.

위에서 구한 feature들은 모두 음성 데이터를 활용하는 다양한 작업(사람 음성 분류, 음성을 통한 감정 분류 등)에서 모델 학습에 사용할 수 있는 feature들입니다.
각각의 **feature들에 따른 모델의 정확도가 얼마나 차이**나는지를 확인해봅시다.

- 베이스라인의 분류기는 **RandomForestClassifier**를 사용합니다.
- **random_state에 따른 성능 차이가 발생하오니 반드시 random_state를 1로 고정해주시길 바랍니다.**

In [41]:
y_train

array(['disgust', 'surprised', 'disgust', ..., 'calm', 'fearful',
       'neutral'], dtype='<U9')

In [42]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
y_train=le.fit_transform(y_train)

In [43]:
y_train

array([2, 7, 2, ..., 1, 3, 5])

In [44]:
#RandomForestClassifier로 음성 감정 분류 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
sample = pd.read_csv(join(DATA_PATH,'sample_submit.csv'))

for feature_name in train_data.keys():
    
    rf=RandomForestClassifier(random_state=1)
    X_train=[]
    x_test=[]
    
    X_train=(train_data[feature_name])
    x_test=((test_data[feature_name]))
    
    X_train=np.asarray(X_train)
    x_test=np.asarray(x_test)
    
    
    print(X_train.shape)
    print(y_train.shape)
    
    rf.fit(X_train,y_train)
    predict=rf.predict(x_test)
    
    
    
    
    #Sample submit file 저장
    sample['emotion'] = le.inverse_transform(predict)
    sample.to_csv(join(feature_name+'.csv'),index=False,header=True)


(1008, 257)
(1008,)
(1008, 128)
(1008,)
(1008, 20)
(1008,)
